# Import the necessary libraries

In [51]:
import os
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm, trange

# Create the necessary folders to store the datasets if not already there

In [4]:
# Folder Containing All Data
DATA_PATH                  = os.path.join(os.getcwd(), 'data')
# Raw Data Folders
FULL_ARGUSII_RAW_DATA_PATH = os.path.join(DATA_PATH, 'MNIST', 'percept')
MNIST_SUBSET_RAW_DATA_PATH = os.path.join(DATA_PATH, 'MNIST_2000_400', 'percept')
# Datasets Out Folders
FULL_ARGUSII_OUT_DATA_PATH = os.path.join(DATA_PATH, 'MNIST', 'processed')
MNIST_SUBSET_OUT_DATA_PATH = os.path.join(DATA_PATH, 'MNIST_2000_400', 'processed')

# Create the Folders IF Not Already There
if not os.path.exists(FULL_ARGUSII_OUT_DATA_PATH):
    os.makedirs(FULL_ARGUSII_OUT_DATA_PATH)
if not os.path.exists(MNIST_SUBSET_OUT_DATA_PATH):
    os.makedirs(MNIST_SUBSET_OUT_DATA_PATH)

In [99]:
from torchvision import transforms
import pickle

def create_dataset(path, out_path, save=True):
    # function to "create" one sample from its name and path
    def one_sample(path, sample_name):
        # keep only those with size greater than zero '0'
        if os.path.getsize(os.path.join(path, sample_name)) <= 0:
            return None

        # extract the sample labels from each name
        sample_label = int(sample_name.split('-')[1].split('.')[0])
        # load sample data
        sample_data = torch.load(os.path.join(path, sample_name))

        #until I change the way it's saved
        sample_data = (sample_data - sample_data.min())/(sample_data.max() - sample_data.min())

        # convert data sample to PIL image
        sample_data = transforms.ToPILImage()(sample_data)

        return sample_data, sample_label

    # function to "create" a samples list from a folder path
    def multiple_samples(path):
        print(f"Create samples list from path: {path}")
        samples_names  = os.listdir(path)
        samples_labels = []
        samples_data   = []
        for sample_name in tqdm(samples_names):
            sample = one_sample(path, sample_name)
            if sample is None:
                continue
            sample_data, sample_label = sample
            samples_labels.append(sample_label)
            samples_data.append(sample_data)

        return samples_data, samples_labels

    # create path if not already there
    if not os.path.exists(out_path):
        os.makedirs(out_path)

#     # list the names of each sample in the train/test folders
#     train_samples_names = os.listdir(os.path.join(path, 'train'))
#     test_samples_names  = os.listdir(os.path.join(path, 'test'))

    train_samples_data, train_samples_labels = multiple_samples(os.path.join(path, 'train'))
    test_samples_data,  test_samples_labels  = multiple_samples(os.path.join(path, 'test' ))

#     for train_sample_name in tqdm(train_samples_names):
#         train_sample_data, train_sample_label = one_sample(os.path.join(path, 'train'), train_sample_name)
#         train_samples_labels.append(train_sample_label)
#         train_samples_data.append(train_sample_data)

#     # keep only those with size greater than zero '0'
#     train_samples_names = [train_samples_name
#                            for train_samples_name in train_samples_names 
#                            if os.path.getsize(os.path.join(path, 'train', train_samples_name)) > 0]      

#     test_samples_names  = [test_samples_name
#                            for test_samples_name  in test_samples_names 
#                            if os.path.getsize(os.path.join(path, 'test' , test_samples_name )) > 0]      
#     print(1)

#     # extract the samples labels from their name
#     train_samples_labels = [int(train_samples_name.split('-')[1].split('.')[0])
#                             for train_samples_name in train_samples_names]
#     test_samples_labels  = [int(test_samples_name.split('-')[1].split('.')[0])
#                             for test_samples_name  in test_samples_names]
#     print(1)

#     # convert labels to torch.Tensor
#     train_samples_labels = torch.Tensor(train_samples_labels)
#     test_samples_labels  = torch.Tensor(test_samples_labels)
#     print(1)
    if save:
        # save labels Tensors in their respective files
        torch.save(train_samples_labels, os.path.join(out_path, 'train_set_labels.pt'))
        torch.save(test_samples_labels,  os.path.join(out_path, 'test_set_labels.pt'))

        # save data list in their respective pickles
        with open(os.path.join(out_path, 'train_set_data.pk'), "wb") as f:
            pickle.dump(train_samples_data, f)

        with open(os.path.join(out_path, 'test_set_data.pk' ), "wb") as f:
            pickle.dump(test_samples_data,  f)

#     torch.save(train_samples_labels, os.path.join(out_path, 'train_set_labels.pt'))
#     torch.save(test_samples_labels,  os.path.join(out_path, 'test_set_labels.pt'))

#     print(1)

#     # load each sample to a list
#     train_samples_data = [torch.load(os.path.join(path, 'train', train_samples_name))
#                           for train_samples_name in train_samples_names]
#     test_samples_data  = [torch.load(os.path.join(path, 'test',  test_samples_name))
#                           for test_samples_name  in test_samples_names]
#     print(1)

#     # convert each data sample to PIL image
#     train_samples_data = [transforms.ToPILImage()(train_sample_data)
#                           for train_sample_data in train_samples_data]
#     test_samples_data  = [transforms.ToPILImage()(test_sample_data)
#                           for test_sample_data  in test_samples_data]
#     print(1)

#     train_samples_data = torch.stack(train_samples_data)
#     test_samples_data  = torch.stack(test_samples_data)

#     torch.save(train_samples_data, os.path.join(out_path, 'train_set_data.pt'))
#     torch.save(test_samples_data,  os.path.join(out_path, 'test_set_data.pt'))

    return ((train_samples_data, train_samples_labels),
            (test_samples_data, test_samples_labels))

In [14]:
train_samples_names = os.listdir(os.path.join(MNIST_SUBSET_RAW_DATA_PATH,
                                              os.listdir(MNIST_SUBSET_RAW_DATA_PATH)[0],
                                              'train'))

In [36]:
sample = torch.load(os.path.join(MNIST_SUBSET_RAW_DATA_PATH,
                                 os.listdir(MNIST_SUBSET_RAW_DATA_PATH)[0],
                                 'train',
                                 train_samples_names[0]))

In [68]:
implant_model_names = os.listdir(MNIST_SUBSET_RAW_DATA_PATH)

In [67]:
for implant_model_name in implant_model_names:
    create_dataset(os.path.join(MNIST_SUBSET_RAW_DATA_PATH, implant_model_name),
                   os.path.join(MNIST_SUBSET_OUT_DATA_PATH, implant_model_name))

In [100]:
(a,b), (c,d) = create_dataset(os.path.join(FULL_ARGUSII_RAW_DATA_PATH, 'AxonMapModel-ArgusII'),
                   os.path.join(FULL_ARGUSII_OUT_DATA_PATH, 'AxonMapModel-ArgusII'))

Create samples list from path: C:\Users\el16077\OneDrive - central.ntua.gr\ΕΜΠ\8ο εξάμηνο\Τεχνολογίες Κινητής & Ηλεκτρονικής Υγείας\Εξαμηνιαία\data\MNIST\percept\AxonMapModel-ArgusII\train



Create samples list from path: C:\Users\el16077\OneDrive - central.ntua.gr\ΕΜΠ\8ο εξάμηνο\Τεχνολογίες Κινητής & Ηλεκτρονικής Υγείας\Εξαμηνιαία\data\MNIST\percept\AxonMapModel-ArgusII\test


In [107]:
a[2554]

In [91]:
a[0]
import numpy as np
ar = np.array(a[0])
ar = 255 - 1*(ar-ar.min())*(255/(ar.max()-ar.min()))
ar = ar.astype(np.uint8)

print(ar.min())
print(ar.max())
Image.fromarray(ar)

0
255
